In [2]:
import pandas as pd
import librosa
import torch
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
import IPython.display as display
import torchaudio
import MeCab
import re
import pykakasi
from tqdm.notebook import tqdm_notebook
from datasets import load_dataset, Dataset
import requests
import time
from datetime import timedelta

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
#pykakasi has deprecation warning and i don't know how to fix 

In [7]:
import os
pickle_folder_path = './livetranslator/uncleaned_pickle'
output_path = './livetranslator/pickle/cleaned.pkl'

In [12]:
def remove_unreadable(pickle_file_list) :
  dataframe_temp_list = []
  for pickle_file in pickle_file_list :
    temp_df = pd.read_pickle(pickle_file)
    dataframe_temp_list.append(temp_df)
  dataframe = pd.concat(dataframe_temp_list)
  err, pss = 0, 0
  for index, row in tqdm_notebook(dataframe.iterrows(), total = len(dataframe), desc = 'Detecting audio that cannot be opened by torchaudio : '):
      try : 
        speech_array, sampling_rate = torchaudio.load(row["path"])
      except :
        dataframe.drop(index, axis = 0, inplace = True) #remove audio if it cannot be opened
        #failed remove 
        err += 1
        continue
      pss += 1
      #pass
  print(f'Finished removing corrupted files < Count Error : {err} // Pass : {pss} -- Total {err+pss} >')
  dataframe.reset_index(drop=True)
  dataset = Dataset.from_pandas(dataframe).remove_columns(['__index_level_0__'])
  return dataframe, dataset

In [28]:
input_list = [x for x in os.listdir(pickle_folder_path)]
dataframe, dataset = remove_unreadable(input_list)
dataframe.to_pickle(output_path)

Detecting audio that cannot be opened by torchaudio :   0%|          | 0/108 [00:00<?, ?it/s]

0 : 108, total = 108


Parameter 'function'=<function preprocessData at 0x7fd9fb687290> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/108 [00:00<?, ?ex/s]

Detecting speech using Wav2Vec2 ::   0%|          | 0/108 [00:00<?, ?it/s]